In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
from itertools import chain

plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_Jul14_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [4]:
hists = {}

hists["hist_particle_pdgID"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_particle_ke"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 10, 600)
                            )

In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    sim_particle = ldmx_events['Sim_PNParticle']
    sim_particle_n = ldmx_events['n']['Sim_PNParticle']
    
    #n2_mask = sim_particle_n >= 2
    
    sorted_e = ak.sort(sim_particle.e, axis=-1, ascending=False, stable=True, highlevel=True, behavior=None)
    
    saved_e = []
    for element in sorted_e:
        if len(element) >= 2:
            x = []
            x.append(ak.to_numpy(element[0]))
            x.append(ak.to_numpy(element[1]))
            #print(x)
            saved_e.append(x)
        elif len(element) == 1:
            saved_e.append([np.array(ak.to_numpy(element))])
        else:
             saved_e.append([np.array([])])
            #print('empty')
            
               
        
   # print(len(sim_particle.e))
   # print(len(sorted_e))
   # print(len(saved_e))
   # print(sim_particle.e)
   # print(sorted_e)
   # print(saved_e)
    
    hist_data = [x]
    for x in list(chain.from_iterable(saved_e)):
        if type(x) == type([]):
            if len(x) != 0:
                hist_data.append(x[0])
        else:
            hist_data.append(x)
    print(hist_data)
    
    
    hists["hist_particle_ke"].fill(
                                     e=hist_data,
                                     ) 
    
    masks_id = {'Kaon1': 321,
                'Kaon2': 310,
                'Kaon3': 130, 
                'Kaon4':-321,
                'Electron': 11,
                'Positron': -11,
                'Pion': 211,
                'Pi-': -211,
                'Proton':2212,
                'Neutron':2112,
                'Sigma': 3112 or 3212,
                'Lambda': 3122,
               }
    masses =   {'Kaon1': 493.677,
                'Kaon2': 493.677,
                'Kaon3': 493.677,
                'Kaon4': 493.677,
                'Electron': 0.511,
                'Positron': 0.511,
                'Pion': 139.57039,
                'Pi-': 139.57039,
                'Proton': 939.565,
                'Neutron': 939.565,
                'Sigma': 1189.36,
                'Lambda': 1115.6,
                }
    
    #particle_vector = get_vector(sim_particle)
    
    #for part,pmask in masks_id.items():
    #    part_mask = (sim_particle.pdgID == pmask)
    #    hists["hist_particle_pdgID"].fill(Ptype= part,
    #                                 e=flat(sim_particle.pdgID[part_mask]),
    #                                 )  
    #    hists["hist_particle_ke"].fill(Ptype= part,
    #                                 e=flat(particle_vector.t[part_mask]) - masses[part],
    #                                 ) 
    

In [6]:
nchunk = 0
#total_particle = 0
#total_electron = 0
#total_sigma    = 0
#total_kaon     = 0
#total_pion     = 0
#total_proton   = 0
#total_neutron  = 0
#total_lambda   = 0

for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

    
#print('All Particles', total_particle, 
#'Electron', total_electron, 
#'Sigma', total_sigma,
#'Kaon', total_kaon,
#'Pion', total_pion,
#'Proton', total_proton,
#'Neutron', total_neutron,
#'Lambda', total_lambda)
    

process 1
[[17.551942825317383, 13.431037902832031], array([], dtype=float64), array([1.024438], dtype=float32), 1262.484130859375, 1196.74560546875, array([], dtype=float64), array([], dtype=float64), 1346.1702880859375, 1243.20703125, array([], dtype=float64), 2005.8555908203125, 1054.5421142578125, array([], dtype=float64), array([], dtype=float64), 1216.7352294921875, 1071.7135009765625, 1561.0733642578125, 1211.5234375, array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), 1742.9017333984375, 1662.3291015625, array([], dtype=float64), 1969.5167236328125, 1327.0130615234375, array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), 1542.8331298828125, 1244.79931640625, 1725.51123046875, 1325.20361328125, array([], dtype=float64), array([0.55385804], dtype=float32), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([5.873372], dtype=float32), array([], dtype=float64), array([], dt

TypeError: Request bin indices with a identifier or 1-D array only

In [ ]:
#48 kminus
print(hists["hist_particle_pdgID"])
fig, ax = plt.subplots(1,2, figsize=(24,10), constrained_layout=True)
hist.plot1d(hists["hist_particle_pdgID"],ax=ax[0],clear=False);
leg = ax[0].legend([r'$e^-$', r'$K^+$', r'$K_s$', r'$K_L$', r'$K^-$', r'$\Lambda$',r'$n^0$', r'$\pi^-$', r'$\pi^+$', r'$e^+$', r'$p^+$', r'$\Sigma$' ])


#hist.plot1d(hists["hist_particle_e"],ax=ax[1],clear=False);
#leg = ax[1].legend([r'$K_L$ with $\theta$<40$^\circ$', r'$K_s$ with $\theta$<40$^\circ$'])

hist.plot1d(hists["hist_particle_ke"],ax=ax[1],clear=False);
leg = ax[1].legend([r'$e^-$', r'$K^+$', r'$K_s$', r'$K_L$', r'$K^-$', r'$\Lambda$',r'$n^0$', r'$\pi^-$', r'$\pi^+$', r'$e^+$', r'$p^+$', r'$\Sigma$' ])
